<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/3_Finans_i%C3%A7in_Python_SMA_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Kütüphanelerin yüklenmesi
!pip install git+https://github.com/rongardF/tvdatafeed
!pip install tradingview-screener
!pip install backtesting


#kütüphanelerin çağırılması
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from backtesting import Backtest, Strategy

#Tradingview Kütüphanesini açıyoruz.
tv = TvDatafeed()

#SMA hesaplamasını tanımlıyoruz.
def sma(series, length):
    calc = series.rolling(window=length).mean()
    return calc

#Her Zaman kullanacağımız Stratejimizi hazırlıyoruz.
class Strategy(Strategy):
    def init(self):
        pass
    def next(self):
        if self.data['Entry'] == True and not self.position:
            self.buy()
        elif self.data['Exit'] == True:
            self.position.close()

#Hisse Adlarını Tanımlıyoruz.
Hisse = ['AKBNK','ALARK','ASELS','ASTOR','BIMAS',
	'BRSAN','DOAS','EKGYO','ENKAI','EREGL',
	'FROTO','GARAN','GUBRF','HEKTS','ISCTR',
	'KCHOL','KONTR','KOZAL','KRDMD','OYAKC',
	'PETKM','PGSUS','SAHOL','SASA','SISE',
	'TCELL','THYAO','TOASO','TUPRS','YKBNK']

# BIST30 sonuçları Arşivlemek için İki Adet Boş dataframe yapısı oluşturuyoruz.
df_bist30_RE = pd.DataFrame(columns=['Hisse Adı','SMA Kısa', 'SMA Uzun','Getiri Yüzdesi'])
df_bist30_WR = pd.DataFrame(columns=['Hisse Adı','SMA Kısa', 'SMA Uzun','Kazanma Oranı'])

for z in range(len(Hisse)):
    #Hisse Verilerini Tradingview Kütüphanesinden çekiyoruz.
    print(Hisse[z])
    data = tv.get_hist(symbol=Hisse[z], exchange='BIST', interval=Interval.in_daily, n_bars=1000)
    #data başlıklarını yeniden isimlendiriyoruz
    data.rename(columns={'open': 'Open', 'high': 'High',
                          'low': 'Low',
                          'close': 'Close',
                          'volume': 'Volume'},
                          inplace=True)

    #Sonuçları Arşivlemek için İki Adet Boş dataframe yapısı oluşturuyoruz.
    df_RE = pd.DataFrame(columns=['SMA Kısa', 'SMA Uzun','Getiri Yüzdesi'])
    df_WR = pd.DataFrame(columns=['SMA Kısa', 'SMA Uzun','Kazanma Oranı'])

    for i in range(5, 51):
        data['SMA_L1'] = sma(data['Close'], i)
        for j in range(i+1, 51):
            data['SMA_L2'] = sma(data['Close'], j)
            data['Entry'] = data['SMA_L1'] > data['SMA_L2']
            data['Exit'] = data['SMA_L1'] < data['SMA_L2']

            # Backtest yapımızı çağırıyoruz ve sonuçları inceliyoruz.
            bt = Backtest(data, Strategy, cash=100000, commission=0.002)
            Stats = bt.run()
            RE = round(Stats.loc['Return [%]'], 2)
            WR = round(Stats.loc['Win Rate [%]'], 2)

            df_RE.loc[len(df_RE)] = [i,j,RE]
            df_WR.loc[len(df_WR)] = [i,j,WR]

    #Sonuçları yazdırma.
    df_RE = df_RE.sort_values(by='Getiri Yüzdesi', ascending=False)
    df_WR = df_WR.sort_values(by='Kazanma Oranı', ascending=False)
    RE_list = df_RE.iloc[0].tolist()
    WR_list = df_WR.iloc[0].tolist()
    df_bist30_RE.loc[len(df_bist30_RE)] = [Hisse[z]]+RE_list
    df_bist30_WR.loc[len(df_bist30_WR)] = [Hisse[z]]+WR_list

print(df_bist30_RE)
print(df_bist30_WR)

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-iquoi7ie
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-iquoi7ie
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


AKBNK
ALARK
ASELS
ASTOR
BIMAS
BRSAN
DOAS
EKGYO
ENKAI
EREGL
FROTO
GARAN
GUBRF
HEKTS
ISCTR
KCHOL
KONTR
KOZAL
KRDMD
OYAKC
PETKM
PGSUS
SAHOL
SASA
SISE
TCELL
THYAO
TOASO
TUPRS
YKBNK
   Hisse Adı  SMA Kısa  SMA Uzun  Getiri Yüzdesi
0      AKBNK      18.0      22.0          805.35
1      ALARK       5.0      38.0         3001.68
2      ASELS      44.0      48.0          515.86
3      ASTOR      15.0      17.0          574.60
4      BIMAS      17.0      25.0          545.60
5      BRSAN      22.0      36.0         5962.99
6       DOAS      10.0      47.0         2300.82
7      EKGYO       5.0       9.0          481.36
8      ENKAI      47.0      49.0          348.80
9      EREGL      21.0      28.0          495.96
10     FROTO      32.0      34.0          921.71
11     GARAN      22.0      33.0         1029.57
12     GUBRF       5.0      11.0          902.66
13     HEKTS      28.0      33.0         1663.04
14     ISCTR       7.0      48.0         1081.20
15     KCHOL      45.0      49.0       